<h2>必要なモジュールのインポート</h2>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/titanic"))

# Any results you write to the current directory are saved as output.

['train.csv', 'gender_submission.csv', 'test.csv']


In [2]:
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')
result_df = test_df[['PassengerId']]

In [3]:
train_df[['Ticket']]

,Ticket
0,A/5 21171
1,PC 17599
2,STON/O2. 3101282
3,113803
4,373450
...,...
886,211536
887,112053
888,W./C. 6607
889,111369


In [4]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
#Check Major prefixes in train columns
major_prefix = train_df['Name'].apply(lambda x: x.split(', ')[1].split('.')[0]).value_counts()[:5]
major_prefix_lst = list(major_prefix.index)
print(major_prefix_lst)

In [ ]:
test_df['Name'].apply(lambda x: x.split(', ')[1].split('.')[0]).value_counts()

In [ ]:
'''arr_cabin = train_df['Cabin'].unique()
initial_lst = []
for i in range(1, len(arr_cabin)):
    initial_lst.append(arr_cabin[i][0])
print(initial_lst)
#print([cabin[0]for cabin in arr_cabin])'''

<h2>**ターゲットとなる値をチェック**</h2>

In [ ]:
train_df['Survived'].value_counts()

In [ ]:
train_df['Cabin'].isnull().sum() / len(train_df)

In [ ]:
train_df['Survived'].plot.hist()

<h2>**NaNの値をチェック**</h2>

In [ ]:
train_df.isnull().sum()

In [ ]:
def get_missing_columns(df):
    mis_vals = df.isnull().sum()
    mis_vals_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_vals, mis_vals_percent], axis=1)
    mis_val_ren_table = mis_val_table.rename(columns = {0: 'number of missing values', 
                                                       1: '% of total values'})
    mis_val_ren_table = mis_val_ren_table[mis_val_ren_table.iloc[:, 1]!=0].sort_values('% of total values', ascending=False)
    
    return mis_val_ren_table

In [ ]:
get_missing_columns(train_df)

In [ ]:
#乗組員だったのかを区分するカラムを追加
def add_has_cabin_num(df):
    df['has Cabin Number'] = df['Cabin'].notnull().apply(lambda x: int(x))
add_has_cabin_num(train_df)
train_df.head()

In [ ]:
add_has_cabin_num(test_df)

In [ ]:
test_df.head()

In [ ]:
#年齢による区分。一旦不要で。
'''
def add_age_category(df):
    age_category_lst = list()
    import math
    for i in range(len(train_df)):
        age = train_df.iloc[i,5]
        if math.isnan(age):
            age_category_lst.append('nan')
        elif age <5:
            age_category_lst.append('baby')
        elif age <10:
            age_category_lst.append('child')
        elif age < 20:
            age_category_lst.append('10s')
        elif age < 30:
            age_category_lst.append('20s')
        elif age < 40:
            age_category_lst.append('30s')
        elif age < 50:
            age_category_lst.append('40s')
        elif age < 60:
            age_category_lst.append('50s')
        else:
            age_category_lst.append('senior')
  
    df['age_category'] = age_category_lst
#train_df'''

In [ ]:
'''add_age_category(train_df)
train_df.head()'''

In [ ]:
#敬称情報を作る関数
def convert_prefix(x):
    prefix = x.split(', ')[1].split('.')[0] 
    if prefix not in major_prefix_lst:
        prefix = 'Others'
    return prefix

In [ ]:
#敬称の情報を追加
def add_prefix(df):
    df['prefix'] = df['Name'].apply(convert_prefix)
add_prefix(train_df)
train_df.head()

In [ ]:
add_prefix(test_df)
test_df.head()

In [ ]:
#家族総数カラムの追加
train_df['family_size'] = train_df['SibSp'] + train_df['Parch']

In [ ]:
train_df.head()

In [ ]:
test_df['family_size'] = test_df['SibSp'] + test_df['Parch']
test_df.head()

In [ ]:
train_df['Cabin'].unique()

In [ ]:
def cre_cabin_ini(col):
    if pd.isnull(col):
        ini = 'NAN'
    else:
        ini = col[0]
    return ini

In [ ]:
#train_df['Cabin_initial'] = train_df['Cabin'].apply(cre_cabin_ini)
#train_df.head()

In [ ]:
#test_df['Cabin_initial'] = test_df['Cabin'].apply(cre_cabin_ini)
#test_df.head()

In [ ]:
train_df.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], inplace=True, axis=1)
train_df.head()

In [ ]:
test_df.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], inplace=True, axis=1)
test_df.head()

In [ ]:
X_train = train_df[['Pclass','Sex', 'SibSp', 'Parch', 'Fare', 'Embarked', 'has Cabin Number', 'prefix', 'family_size']]
y_train = train_df[['Survived']]

In [ ]:
X_test = test_df[['Pclass','Sex', 'SibSp', 'Parch', 'Fare', 'Embarked', 'has Cabin Number', 'prefix', 'family_size']]

<h2>データの理解</h2>

In [ ]:
#seabornのcount plotを見てみる。
sns.countplot(x='Pclass', data=train_df, hue='Survived')

In [ ]:
sns.countplot(x='Sex', data=train_df, hue='Survived')

In [ ]:
sns.countplot(x='Embarked', data=train_df, hue='Survived')

In [ ]:
sns.countplot(x='has Cabin Number', data=train_df, hue='Survived')

In [ ]:
train_df[train_df['Age'].notnull()]['Age']

In [ ]:
g = sns.FacetGrid(train_df, hue='Survived', height=10)
g.map(sns.distplot, 'Fare', kde=False)
g.add_legend()

In [ ]:
g = sns.FacetGrid(train_df, hue='Survived', height=5)
g.map(sns.distplot, 'Age')
g.add_legend()

In [ ]:
sns.countplot(x='SibSp', data=train_df, hue='Survived')

In [ ]:
sns.countplot(x='Parch', data=train_df, hue='Survived')

In [ ]:
sns.countplot('family_size', data=train_df, hue='Survived')

<h2>Label Encoding</h2>

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
for col in ['Sex', 'Embarked', 'prefix']:
    le = LabelEncoder()
    le.fit(X_train[col].fillna('NA'))
    
    X_train[col] = le.transform(X_train[col].fillna('NA'))
    X_test[col] = le.transform(X_test[col].fillna('NA'))

<h2>XGBoostingを用いたモデル</h2>

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier(n_estimators=20, random_state=71)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
pred = model.predict_proba(X_test)[:, 1]

In [ ]:
pred_label = np.where(pred > 0.5, 1, 0)

In [ ]:
result_df['Survived'] = pred_label

In [ ]:
result_df.to_csv('submit.csv', index=False)

In [ ]:
#パラメーターを最適化しながら実施
